# Лабораторная работа №2

### Используемые библиотеки

In [97]:
import pandas as pd
import itertools
import mlxtend.frequent_patterns as fp
import mlxtend.preprocessing as pp

### Исходные данные

`data1.csv`:
```csv
tid,itemset
t1,ABCD
t2,ACDF
t3,ACDEG
t4,ABDF
t5,BCG
t6,DFG
t7,ABG
t8,CDFG
```

`data2.csv`:
```csv
tid,itemset
1,2 3 6 7
2,1 3 4 8 11
3,3 9 11
4,1 5 6 7
5,1 3 8 10 11
6,3 5 7 9 11
7,4 6 8 10 11
8,1 3 5 8 11
```

## Загрузка данных

In [98]:
frame = pd.read_csv('data1.csv').apply(lambda x: x.apply(list) if x.name == 'itemset' else x)
print(f"Исходный набор данных:")
frame

Исходный набор данных:


,tid,itemset
0,t1,"[A, B, C, D]"
1,t2,"[A, C, D, F]"
2,t3,"[A, C, D, E, G]"
3,t4,"[A, B, D, F]"
4,t5,"[B, C, G]"
5,t6,"[D, F, G]"
6,t7,"[A, B, G]"
7,t8,"[C, D, F, G]"


In [99]:
te = pp.TransactionEncoder()
te_ary = te.fit_transform(frame['itemset'].tolist())
frame = pd.DataFrame(te_ary, columns=te.columns_)
frame

,A,B,C,D,E,F,G
0,True,True,True,True,False,False,False
1,True,False,True,True,False,True,False
2,True,False,True,True,True,False,True
3,True,True,False,True,False,True,False
4,False,True,True,False,False,False,True
5,False,False,False,True,False,True,True
6,True,True,False,False,False,False,True
7,False,False,True,True,False,True,True


In [100]:
results = fp.apriori(frame, min_support=3.0/8.0, use_colnames=True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
results

,support,itemsets,length
0,0.625,(A),1
1,0.500,(B),1
2,0.625,(C),1
3,0.750,(D),1
4,0.500,(F),1
5,0.625,(G),1
6,0.375,"(B, A)",2
7,0.375,"(C, A)",2
8,0.500,"(D, A)",2
9,0.500,"(C, D)",2


In [101]:
results = fp.fpgrowth(frame, min_support=2.0/8.0, use_colnames=True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
results

,support,itemsets,length
0,0.750,(D),1
1,0.625,(C),1
2,0.625,(A),1
3,0.500,(B),1
4,0.500,(F),1
5,0.625,(G),1
6,0.500,"(C, D)",2
7,0.375,"(C, G)",2
8,0.250,"(C, G, D)",3
9,0.500,"(D, A)",2


## Загрузка данных

In [102]:
frame = pd.read_csv('data2.csv').apply(lambda x: x.apply(str.split) if x.name == 'itemset' else x)
print(f"Исходный набор данных:")
frame

Исходный набор данных:


,tid,itemset
0,1,"[2, 3, 6, 7]"
1,2,"[1, 3, 4, 8, 11]"
2,3,"[3, 9, 11]"
3,4,"[1, 5, 6, 7]"
4,5,"[1, 3, 8, 10, 11]"
5,6,"[3, 5, 7, 9, 11]"
6,7,"[4, 6, 8, 10, 11]"
7,8,"[1, 3, 5, 8, 11]"


In [103]:
field = [item for sub in [itertools.combinations(range(1, 12), i) for i in range(1, 12)] for item in sub]
len(field)

2047

In [104]:
def process_leaves(leaves):
    res = []
    for leaf in leaves:
        leaf = int(leaf)
        if leaf in (12, 5):
            res += [leaf, 14]
        elif leaf in (7, 13, 11):
            res += [leaf, 15]
        elif leaf in (2, 3, 4):
            res += [leaf, 12, 14]
        elif leaf in (8, 9, 10):
            res += [leaf, 13, 15]
        else:
            res += [leaf]
    return list(set(res))

print(f"Набор высоких в таксометрии элементов:")
frame = frame.apply(lambda x: x.apply(process_leaves) if x.name == 'itemset' else x)
frame

Набор высоких в таксометрии элементов:


,tid,itemset
0,1,"[2, 3, 6, 7, 12, 14, 15]"
1,2,"[1, 3, 4, 8, 11, 12, 13, 14, 15]"
2,3,"[3, 9, 11, 12, 13, 14, 15]"
3,4,"[1, 5, 6, 7, 14, 15]"
4,5,"[1, 3, 8, 10, 11, 12, 13, 14, 15]"
5,6,"[3, 5, 7, 9, 11, 12, 13, 14, 15]"
6,7,"[4, 6, 8, 10, 11, 12, 13, 14, 15]"
7,8,"[1, 3, 5, 8, 11, 12, 13, 14, 15]"


In [105]:
te = pp.TransactionEncoder()
te_ary = te.fit_transform(frame['itemset'].tolist())
frame = pd.DataFrame(te_ary, columns=te.columns_)

results = fp.apriori(frame, min_support=7.0/8.0, use_colnames=True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
results

,support,itemsets,length
0,0.875,(12),1
1,1.000,(14),1
2,1.000,(15),1
3,0.875,"(12, 14)",2
4,0.875,"(12, 15)",2
5,1.000,"(14, 15)",2
6,0.875,"(12, 14, 15)",3
